In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd
import re,string,unicodedata

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from gensim.utils import simple_preprocess 
from gensim.models import Phrases, Word2Vec
from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Load the IMDb dataset from the CSV file
imdb_df = pd.read_csv('/irony-labeled.csv', header=None, names=['comment_text', 'label'])
imdb_df["label"] = np.where(imdb_df["label"] == "-1", 1, 0)

gen_df = pd.read_csv('/GEN-sarc-notsarc.csv', header=None, names=['class', 'text'])
gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))

gen_df["class"] = np.where(gen_df["class"] == "notsarc", 0, 1)

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Preprocess the dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

<ipython-input-2-54c2b024cd1e>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
<ipython-input-2-54c2b024cd1e>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))


In [3]:
imdb_df['comment_text']=imdb_df['comment_text'].apply(denoise_text)
gen_df['text']=gen_df['text'].apply(denoise_text)

<ipython-input-2-54c2b024cd1e>:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [4]:
words, imdb_words, gen_words = [], [], []
for i in imdb_df.comment_text.values:
    words.append(i.split())
    imdb_words.append(i.split())
words=words[1:]

for i in gen_df.text.values:
    words.append(i.split())
    gen_words.append(i.split())

In [5]:
len(words)

11336

In [6]:
imdb_wv_model = Word2Vec(words, vector_size=100, window=3, min_count=1, workers=16)

In [7]:
print(imdb_wv_model)

Word2Vec<vocab=53846, vector_size=100, alpha=0.025>


In [8]:
print(len(words))

11336


In [9]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(53846)
tokenizer.fit_on_texts(words)
text = tokenizer.texts_to_sequences(words)
text = keras.utils.pad_sequences(text, 50)

print(text[:2])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0   840   610  7758  5688   668   192     3  5689    86
    359  1035  4569 19649 19650 19651   100  2416  5058  1239  4570 19652
    289     8  5690   777   206   668 12542   381  1688   175  2000   175
    329   516]
 [  528   424 19653   134    26   757  1317  5059   841   117  2292   517
    107  1450 19654  3540   669   311 19655  2293   710  1171   257  9557
   2894     8 19656 19657     8  7759 12543 19658 19659  2293   710  2894
   1567   378    64    76  5060    17  4571  9558    92    27  4158   469
   6613  4571]]


In [39]:
y = imdb_df["label"]

In [11]:
all(y)

False

In [40]:
txt = tokenizer.texts_to_sequences(imdb_words)
txt = keras.utils.pad_sequences(txt, 50)

txt[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,   840,   610,  7758,
         5688,   668,   192,     3,  5689,    86,   359,  1035,  4569,
        19649, 19650, 19651,   100,  2416,  5058,  1239,  4570, 19652,
          289,     8,  5690,   777,   206,   668, 12542,   381,  1688,
          175,  2000,   175,   329,   516]], dtype=int32)

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(txt), np.array(y), train_size=0.8, stratify=y)

In [42]:
import tensorflow as tf

model = keras.models.Sequential([

    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=100,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    keras.layers.Dropout(0.3),
    keras.layers.Conv1D(50, 3, activation='relu', padding='same', strides=1),
    keras.layers.MaxPool1D(2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
    keras.layers.Dropout(0.2),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 50, 100)           5384600   
                                                                 
 dropout_42 (Dropout)        (None, 50, 100)           0         
                                                                 
 conv1d_15 (Conv1D)          (None, 50, 50)            15050     
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 25, 50)           0         
 g1D)                                                            
                                                                 
 dropout_43 (Dropout)        (None, 25, 50)            0         
                                                                 
 flatten_11 (Flatten)        (None, 1250)              0         
                                                     

In [43]:
val_recall = 0.8662 *100
val_precision = 0.8787 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

87.24052266605536

In [24]:
model.save("/cnn-irony.h")

In [44]:
gen_wv_model = Word2Vec(gen_words, vector_size=100, window=3, min_count=1, workers=16)

In [45]:
print(gen_wv_model)

Word2Vec<vocab=46908, vector_size=100, alpha=0.025>


In [48]:
# tokenizer = Tokenizer(46908)
# tokenizer.fit_on_texts(gen_words)
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=100,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    keras.layers.Dropout(0.3),
    keras.layers.Conv1D(50, 3, activation='relu', padding='same', strides=1),
    keras.layers.MaxPool1D(2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
    keras.layers.Dropout(0.2),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y, random_state=42)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
235/235 [==============================] - 26s 104ms/step - loss: 0.4497 - accuracy: 0.8458 - precision_17: 0.8477 - recall_17: 0.9972 - val_loss: 0.4421 - val_accuracy: 0.8472 - val_precision_17: 0.8472 - val_recall_17: 1.0000
Epoch 2/10
235/235 [==============================] - 11s 46ms/step - loss: 0.4351 - accuracy: 0.8474 - precision_17: 0.8474 - recall_17: 1.0000 - val_loss: 0.4166 - val_accuracy: 0.8472 - val_precision_17: 0.8472 - val_recall_17: 1.0000
Epoch 3/10
235/235 [==============================] - 6s 27ms/step - loss: 0.4118 - accuracy: 0.8474 - precision_17: 0.8474 - recall_17: 1.0000 - val_loss: 0.4148 - val_accuracy: 0.8472 - val_precision_17: 0.8472 - val_recall_17: 1.0000
Epoch 4/10
235/235 [==============================] - 5s 22ms/step - loss: 0.3251 - accuracy: 0.8551 - precision_17: 0.8555 - recall_17: 0.9975 - val_loss: 0.4320 - val_accuracy: 0.8387 - val_precision_17: 0.8474 - val_recall_17: 0.9874
Epoch 5/10
235/235 [=============================

In [49]:
val_recall = 0.8752 *100
val_precision = 0.8152 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

84.41351632749645

In [ ]:
!zip -r "/bilstm.zip" "/bert-with-intermediate-task.h"

  adding: bert-with-intermediate-task.h/ (stored 0%)
  adding: bert-with-intermediate-task.h/assets/ (stored 0%)
  adding: bert-with-intermediate-task.h/keras_metadata.pb (deflated 92%)
  adding: bert-with-intermediate-task.h/variables/ (stored 0%)
  adding: bert-with-intermediate-task.h/variables/variables.index (deflated 67%)
  adding: bert-with-intermediate-task.h/variables/variables.data-00000-of-00001 (deflated 48%)
  adding: bert-with-intermediate-task.h/fingerprint.pb (stored 0%)
  adding: bert-with-intermediate-task.h/saved_model.pb (deflated 90%)
